In [1]:
import sys
sys.path.append('../')

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

import QUBO_SVM_functions as qSVM
import kernel_SVM_functions as kSVM
import metric_functions as mf
from kernel_SVM_functions import rbf_kernel
from parameter_tuning import tune_csvm_parameters

from quantum_classifier import QSVMq
from quantum_annealing_functions import QA_cross_validate, QA_calibration, stratified_sample

In [3]:
filepath = '../synth_data/synth_0.3.csv'

data = np.loadtxt(filepath, delimiter = ',')
X = data[:, :-1]
t = data[:, -1]

X_train, X_test, t_train, t_test = train_test_split(X, t, train_size = 0.17, shuffle = True, stratify = t)
t_train = t_train.reshape(-1, 1)
t_test = t_test.reshape(-1, 1)
print(X_train.shape)

(20, 2)


In [9]:
B_values = [2, 3]
K_values = [3, 4]
R_values = [0, 1]
gamma_values = [2, 4]

In [45]:
param_combinations = np.array([[[i, B], [j, K], [k, R], [l, gamma]] for i, B in enumerate(B_values) 
                                                            for j, K in enumerate(K_values) 
                                                            for k, R in enumerate(R_values)
                                                            for l, gamma in enumerate(gamma_values)])

In [42]:
def include_indices(param_combinations, to_skip):
    #Takes in a list of combinations to skip and returns a boolean array of those combinations to include
    indices = []
    for arr in to_skip:
        indices.append(np.array([(param_combinations[i, :, 1] == arr).all() for i in range(param_combinations.shape[0])], dtype=int))

    return ~np.sum(indices, axis = 0).astype(bool)

In [47]:
to_skip = np.array([[2, 3, 1, 2], [2, 4, 0, 4]])
include_index = include_indices(param_combinations, to_skip)
print(param_combinations[include_index, :, 1])

[[2 3 0 2]
 [2 3 0 4]
 [2 3 1 4]
 [2 4 0 2]
 [2 4 1 2]
 [2 4 1 4]
 [3 3 0 2]
 [3 3 0 4]
 [3 3 1 2]
 [3 3 1 4]
 [3 4 0 2]
 [3 4 0 4]
 [3 4 1 2]
 [3 4 1 4]]


In [48]:
from time import time
time()

1689875002.6002078

In [12]:
for (i, B), (j, K), (k, R), (l, gamma) in param_combinations:

    print(B, K, R, gamma, '\n')

2 3 0 2 

2 3 0 4 

2 3 1 2 

2 3 1 4 

2 4 0 2 

2 4 0 4 

2 4 1 2 

2 4 1 4 

3 3 0 2 

3 3 0 4 

3 3 1 2 

3 3 1 4 

3 4 0 2 

3 4 0 4 

3 4 1 2 

3 4 1 4 



In [13]:
sample_df = pd.read_csv('../QA_results/(2, 3, 1, 2)/sample-1/(2, 3, 1, 2)-f1')

In [14]:
sample_df

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,47,48,49,50,51,52,53,chain_break_fraction,energy,num_occurrences
0,0,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0.000000,-1512.308880,72
1,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0.018519,-1512.308880,3
2,2,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0.018519,-1512.308880,1
3,3,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0.018519,-1512.308880,2
4,4,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0.018519,-1512.308880,1
5,5,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0.018519,-1512.308880,2
6,6,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0.018519,-1512.308880,1
7,7,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0.018519,-1512.308880,1
8,8,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0.018519,-1512.308880,1
9,9,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0.018519,-1512.308880,1


In [15]:
svm = QSVMq(2, 3, 1, rbf_kernel, 2)
svm = svm.make_QUBO_problem(X_train, t_train).fit(X_train, t_train, 'nowhere')

In [22]:
Q = svm.Q
q = svm.q
N = Q.shape[0]

qubo = np.triu(Q + np.identity(N) * q)
qubo_dict = {(i, j): qubo[i, j] for i in range(N) for j in range(N)} 

In [9]:
alphas = svm.top_models_arr[0]
svm = svm.set_model(X_train, t_train, alphas)

AssertionError: either t_train or alphas does not match the shape of X_train

In [27]:
print(t_train.shape)
ss = stratified_sample(t_train)
print(X_train[ss].shape, t_train[ss].shape)

(20, 1)
(20, 2) (20, 1)
